In [49]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from modeler.modeler import Modeler as m
from preprocessor.model_preprocessor import ModelPreprocessor as mp
from strategy.quarterly_financial_categorical import QuarterlyFinancialCategorical
from datetime import datetime, timedelta
import numpy as np
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime, timezone
import pickle

In [2]:
market = Market()
sec = SEC("sec")
qfc = QuarterlyFinancialCategorical()

In [3]:
market.connect()
sp5 = market.retrieve_data("sp500")
prices = market.retrieve_data("prices")
market.close()

In [4]:
prices["year"]  = [x.year for x in prices["date"]]
prices["quarter"]  = [x.quarter for x in prices["date"]]

In [5]:
quarterly_grouped = prices.groupby(["year","quarter","ticker"]).max()
quarterly_grouped["category"] = [math.ceil(x / 100 ) * 100 for x in quarterly_grouped["adjclose"]]
quarterly_grouped["category"] = [250 if x > 100 else x for x in quarterly_grouped["category"]]
quarterly_grouped["category"] = [500 if x > 250 else x for x in quarterly_grouped["category"]]
quarterly_grouped["category"] = [1000 if x > 500 else x for x in quarterly_grouped["category"]]
quarterly_grouped["category"] = [2000 if x > 1000 else x for x in quarterly_grouped["category"]]
quarterly_grouped["category"] = [3000 if x > 2000 else x for x in quarterly_grouped["category"]]

In [6]:
quarterly_grouped.reset_index(inplace=True)
groups = quarterly_grouped.merge(sp5.rename(columns={"Symbol":"ticker"}), on = "ticker",how="left")

In [7]:
g = groups[["year","quarter","ticker","adjclose","category","GICS Sector","CIK"]]
g["string_category"] = [str(x) for x in g["category"]] 
g["classification"] = g["string_category"] + g["GICS Sector"]

In [8]:
enc = OneHotEncoder(handle_unknown='ignore')
transformed = [[x] for x in g["classification"]]
encoding = enc.fit_transform(transformed)

In [9]:
df_encoding = pd.DataFrame(encoding.toarray())

In [10]:
for col in df_encoding.columns:
    g[col] = df_encoding[col]

In [73]:
yearly_gap = 1
training_years = 1
fails = []
filings = []
columns = []
sec.connect()
for cik in tqdm(list(g["CIK"].unique())):
    try:
        filing = sec.retrieve_filing_data(int(cik))
        symbols = sp5[sp5["CIK"]==cik]["Symbol"]
        if symbols.index.size > 1:
            ticker = str(list(symbols)[0])
        else:
            ticker = symbols.item()
        drop_columns = ["cik","filed","_id","adsh"]
        funds = filing.copy()
        for column in funds.columns:
            if str(column).islower() and str(column) != "filed":
                drop_columns.append(column)
        funds["filed"] = [datetime.strptime(str(x),"%Y%m%d").replace(tzinfo=timezone.utc) if "-" not in str(x) else \
                            datetime.strptime(str(x).split(" ")[0],"%Y-%m-%d").replace(tzinfo=timezone.utc) for x in funds["filed"]]
        funds["quarter"] = [x.quarter for x in funds["filed"]]
        funds["year"] = [x.year + yearly_gap for x in funds["filed"]]
        funds["ticker"] = ticker
        funds.drop(drop_columns,axis=1,inplace=True)
        qa = funds.copy()
        for col in qa.columns:
            test = qa[col].fillna(-99999)
            availability = 1 - (len([x for x in test if x == -99999]) / qa.index.size)
            if availability < 0.95:
                funds.drop(col,inplace=True,axis=1)
        filings.append(funds)
        columns.append(list(funds.columns))
    except Exception as e:
        print("prep",ticker,str(e))
        fails.append([ticker,str(e)])
sec.close()
try:
    f = pd.concat(filings)
    for col in tqdm(f.columns):
        test = f[col].fillna(-99999)
        availability = len([x for x in test != -99999 if x == True]) / test.index.size
        if availability < 0.7:
            f.drop(col,axis=1,inplace=True)
except Exception as e:
    print("mid",str(e))
try:
    data = f.merge(g.drop(["string_category","classification","adjclose","category","GICS Sector","CIK"],axis=1),on=["year","quarter","ticker"],how="left")
    factors = list(data.columns)
    factors = [x for x in factors if x not in ["quarter","year","ticker"]]
    for i in range(22):
        factors.remove(i)
    for col in factos:
        data[col].replace([np.inf, -np.inf,np.nan,np.NaN],f[col].mean(),inplace=True)
except Exception as e:
    print(str(e))


2012 1 461
2012 2 465
2012 3 465
2012 4 466
2013 1 468
2013 2 472
2013 3 472
2013 4 476
2014 1 476
2014 2 479
2014 3 482
2014 4 483
2015 1 484
2015 2 486
2015 3 488
2015 4 489
2016 1 490
2016 2 490
2016 3 491
2016 4 492
2017 1 492
2017 2 493
2017 3 493
2017 4 493
2018 1 493
2018 2 493
2018 3 493
2018 4 493
2019 1 496
2019 2 497
2019 3 497
2019 4 497
2020 1 499
2020 2 499
2020 3 500
2020 4 500


In [79]:
f = qfc.retrieve_data("categorical_training_data").drop("_id",axis=1)

In [80]:
try:
    data = f.merge(g.drop(["string_category","classification","adjclose","category","GICS Sector","CIK"],axis=1),on=["year","quarter","ticker"],how="left")
except Exception as e:
    print(str(e))

In [101]:
data

,AccumulatedOtherComprehensiveIncomeLossNetOfTax,Assets,AssetsCurrent,CashAndCashEquivalentsAtCarryingValue,EarningsPerShareBasic,EarningsPerShareDiluted,EntityCommonStockSharesOutstanding,IncomeTaxExpenseBenefit,LiabilitiesAndStockholdersEquity,LiabilitiesCurrent,...,12,13,14,15,16,17,18,19,20,21
0,3.050000e+07,4.385600e+10,3.374050e+10,9.051250e+09,2.830000,2.777500,895816758.0,1.065750e+09,4.385600e+10,1.537650e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.600000e+07,4.671150e+10,3.428800e+10,8.220500e+09,5.303333,5.193333,900678473.0,1.951000e+09,4.671150e+10,1.668700e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.400000e+07,5.071350e+10,3.244350e+10,7.995750e+09,3.140000,3.085000,906794589.0,1.192000e+09,5.071350e+10,1.230150e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.400000e+07,4.183600e+10,3.078050e+10,8.220500e+09,6.733333,6.596667,900678473.0,2.723333e+09,4.183600e+10,1.143350e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.010000e+08,5.227900e+10,3.194550e+10,7.905500e+09,4.170000,4.102500,909938383.0,1.456500e+09,5.227900e+10,1.186750e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39449,-3.163333e+09,4.283417e+10,1.412983e+10,2.216667e+09,NaN,NaN,374284600.0,1.845000e+08,4.283417e+10,8.114000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39450,-1.494500e+09,2.220350e+10,5.963500e+09,8.610000e+08,0.285000,0.285000,866158910.0,1.665000e+08,2.220350e+10,4.460000e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39451,-1.366000e+09,2.329100e+10,7.006000e+09,1.530667e+09,0.747500,0.747500,866164968.0,1.687500e+08,2.329100e+10,4.629000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39452,-7.555000e+08,9.605500e+09,5.586000e+09,1.324333e+09,NaN,NaN,433079455.0,1.250000e+08,9.605500e+09,5.364500e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
year_range = range(2012,2019)
year_gap = 1
for year in tqdm(year_range):
    try:
        training_data = data[(data["year"] < year) & (data["year"] >= year - year_gap)]
        factors = list(data.columns)
        factors = [x for x in factors if x not in ["quarter","year","ticker"]]
        for i in range(22):
            factors.remove(i)
        for col in factors:
            training_data[col].replace([np.inf, -np.inf,np.nan,np.NaN],training_data[col].mean(),inplace=True)
        training_data.dropna(inplace=True)
        x = training_data[factors]
        y = training_data[[x for x in range(22)]]
        prediction_data = data[(data["year"]==year)]
        refined_data = {"X":x.reset_index(drop=True),"y":y.reset_index(drop=True)}
        classification_models = m.classification(refined_data.copy(),deep=False,tf=False,multioutput=True)
        models = pd.DataFrame([classification_models])
        model = models["model"].item()
        for col in factors:
            prediction_data[col].replace([np.inf, -np.inf,np.nan,np.NaN],prediction_data[col].mean(),inplace=True)
        prediction_data.dropna(inplace=True)
        predictions = enc.inverse_transform(model.predict(prediction_data[factors]))
        prediction_data["prediction"] = [x[0] for x in predictions]
        sim = prediction_data[["year","quarter","ticker","prediction"]]
        qfc.db.connect()
        qfc.db.store_data("stock_category_sim",sim)
        qfc.db.close()
    except Exception as e:
        print(str(e))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:25<00:00, 12.16s/it]


In [102]:
prediction_data[["year","quarter","ticker"]].head(10)

,year,quarter,ticker
33,2018,1,AAPL
34,2018,2,AAPL
35,2018,3,AAPL
36,2018,4,AAPL
79,2018,1,AAPL
80,2018,2,AAPL
81,2018,3,AAPL
82,2018,4,AAPL
116,2018,1,ABMD
117,2018,2,ABMD
